# Recurrent neural networks

Sa nakaraang module, gumamit tayo ng mas mayamang semantic na representasyon ng teksto, at isang simpleng linear classifier sa ibabaw ng embeddings. Ang ginagawa ng arkitekturang ito ay kunin ang pinagsama-samang kahulugan ng mga salita sa isang pangungusap, ngunit hindi nito isinasaalang-alang ang **pagkakasunod-sunod** ng mga salita, dahil ang operasyon ng pagsasama-sama sa ibabaw ng embeddings ay tinanggal ang impormasyong ito mula sa orihinal na teksto. Dahil hindi kayang i-modelo ng mga modelong ito ang pagkakasunod-sunod ng mga salita, hindi nila masosolusyunan ang mas kumplikado o malalabong gawain tulad ng pagbuo ng teksto o pagsagot sa mga tanong.

Upang makuha ang kahulugan ng pagkakasunod-sunod ng teksto, kailangan nating gumamit ng ibang arkitektura ng neural network, na tinatawag na **recurrent neural network**, o RNN. Sa RNN, ipinapasa natin ang ating pangungusap sa network nang paisa-isang simbolo, at ang network ay gumagawa ng isang **estado**, na pagkatapos ay ipinapasa natin muli sa network kasama ang susunod na simbolo.

Given ang input sequence ng mga token $X_0,\dots,X_n$, ang RNN ay lumilikha ng isang sequence ng mga neural network blocks, at sinasanay ang sequence na ito end-to-end gamit ang back propagation. Ang bawat network block ay tumatanggap ng pares $(X_i,S_i)$ bilang input, at gumagawa ng $S_{i+1}$ bilang resulta. Ang final state $S_n$ o output $X_n$ ay ipinapasa sa isang linear classifier upang makuha ang resulta. Ang lahat ng network blocks ay may parehong weights, at sinasanay end-to-end gamit ang isang back propagation pass.

Dahil ang mga state vectors $S_0,\dots,S_n$ ay ipinapasa sa network, kaya nitong matutunan ang mga sequential dependencies sa pagitan ng mga salita. Halimbawa, kapag ang salitang *not* ay lumitaw sa isang bahagi ng sequence, maaari nitong matutunan na i-negate ang ilang elemento sa loob ng state vector, na nagreresulta sa negation.

> Dahil ang weights ng lahat ng RNN blocks sa larawan ay shared, ang parehong larawan ay maaaring i-representa bilang isang block (sa kanan) na may recurrent feedback loop, na ipinapasa ang output state ng network pabalik sa input.

Tingnan natin kung paano makakatulong ang recurrent neural networks sa pag-classify ng ating news dataset.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Simpleng RNN na tagapag-uri

Sa kaso ng simpleng RNN, ang bawat recurrent unit ay isang simpleng linear na network, na tumatanggap ng pinagsamang input vector at state vector, at gumagawa ng bagong state vector. Kinakatawan ng PyTorch ang unit na ito gamit ang klase na `RNNCell`, at ang mga network ng ganitong mga cell - bilang `RNN` layer.

Para magtakda ng isang RNN na tagapag-uri, una nating gagamitin ang embedding layer upang mabawasan ang dimensionality ng input na bokabularyo, at pagkatapos ay maglalagay ng RNN layer sa ibabaw nito:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Tandaan:** Gumagamit tayo ng untrained embedding layer dito para sa kasimplehan, ngunit para sa mas magagandang resulta, maaari tayong gumamit ng pre-trained embedding layer gamit ang Word2Vec o GloVe embeddings, tulad ng ipinaliwanag sa nakaraang unit. Para mas maintindihan, maaaring i-adapt ang code na ito upang gumana gamit ang pre-trained embeddings.

Sa ating kaso, gagamit tayo ng padded data loader, kaya ang bawat batch ay magkakaroon ng bilang ng mga padded sequence na may parehong haba. Ang RNN layer ay tatanggap ng sequence ng embedding tensors, at magbibigay ng dalawang output:
* $x$ ay isang sequence ng mga output ng RNN cell sa bawat hakbang
* $h$ ay ang huling hidden state para sa pinakahuling elemento ng sequence

Pagkatapos, mag-a-apply tayo ng fully-connected linear classifier upang makuha ang bilang ng klase.

> **Tandaan:** Ang mga RNN ay medyo mahirap i-train, dahil kapag ang mga RNN cell ay na-unroll sa haba ng sequence, ang bilang ng mga layer na kasangkot sa back propagation ay nagiging napakarami. Dahil dito, kailangan nating pumili ng maliit na learning rate, at i-train ang network sa mas malaking dataset upang makakuha ng magagandang resulta. Maaari itong tumagal ng mahabang oras, kaya mas mainam na gumamit ng GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Long Short Term Memory (LSTM)

Isa sa mga pangunahing problema ng klasikong RNNs ay ang tinatawag na **vanishing gradients** na problema. Dahil ang RNNs ay sinasanay mula simula hanggang dulo sa isang back-propagation pass, nahihirapan itong ipasa ang error sa mga unang layer ng network, kaya't hindi natututo ang network ng mga relasyon sa pagitan ng mga malalayong token. Isa sa mga paraan upang maiwasan ang problemang ito ay ang pagpapakilala ng **explicit state management** gamit ang tinatawag na **gates**. Dalawa sa mga pinakakilalang arkitektura ng ganitong uri ay ang **Long Short Term Memory** (LSTM) at **Gated Relay Unit** (GRU).

![Larawan na nagpapakita ng halimbawa ng long short term memory cell](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Ang LSTM Network ay nakaayos sa paraang katulad ng RNN, ngunit may dalawang estado na ipinapasa mula layer patungo sa layer: ang aktwal na estado $c$, at ang nakatagong vector $h$. Sa bawat unit, ang nakatagong vector $h_i$ ay pinagsasama sa input $x_i`, at sila ang nagkokontrol kung ano ang mangyayari sa estado $c` sa pamamagitan ng **gates**. Ang bawat gate ay isang neural network na may sigmoid activation (output sa saklaw na $[0,1]$), na maaaring isipin bilang bitwise mask kapag pinarami sa state vector. Narito ang mga sumusunod na gates (mula kaliwa hanggang kanan sa larawan sa itaas):
* **forget gate** - kinukuha ang nakatagong vector at tinutukoy kung aling mga bahagi ng vector $c` ang kailangan nating kalimutan, at alin ang ipapasa.
* **input gate** - kumukuha ng ilang impormasyon mula sa input at nakatagong vector, at isinasama ito sa estado.
* **output gate** - binabago ang estado sa pamamagitan ng ilang linear layer na may $\tanh$ activation, pagkatapos ay pinipili ang ilang bahagi nito gamit ang nakatagong vector $h_i` upang makabuo ng bagong estado $c_{i+1}`.

Ang mga bahagi ng estado $c` ay maaaring isipin bilang mga flag na maaaring i-on o i-off. Halimbawa, kapag nakatagpo tayo ng pangalang *Alice* sa sequence, maaaring ipalagay na tumutukoy ito sa isang babaeng karakter, at itataas ang flag sa estado na mayroon tayong pangngalang pambabae sa pangungusap. Kapag nakatagpo pa tayo ng mga pariralang *and Tom*, itataas natin ang flag na mayroon tayong pangngalang maramihan. Kaya't sa pamamagitan ng pagmamanipula ng estado, maaari nating subaybayan ang mga gramatikal na katangian ng mga bahagi ng pangungusap.

> **Note**: Isang mahusay na mapagkukunan para sa pag-unawa sa mga detalye ng LSTM ay ang artikulong [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) ni Christopher Olah.

Bagama't maaaring mukhang kumplikado ang panloob na istruktura ng LSTM cell, itinatago ng PyTorch ang implementasyong ito sa loob ng `LSTMCell` class, at nagbibigay ng `LSTM` object upang kumatawan sa buong LSTM layer. Kaya't ang implementasyon ng LSTM classifier ay magiging halos katulad ng simpleng RNN na nakita natin sa itaas:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Mga Naka-pack na Sequence

Sa ating halimbawa, kinailangan nating magdagdag ng zero vectors sa lahat ng sequence sa minibatch. Bagamat nagdudulot ito ng kaunting pag-aaksaya ng memorya, mas kritikal sa RNNs na ang karagdagang RNN cells ay nalilikha para sa mga padded na input, na kasali sa training ngunit walang mahalagang impormasyon na dala. Mas mainam kung ang RNN ay ituturo lamang sa aktwal na haba ng sequence.

Upang magawa ito, isang espesyal na format ng pag-iimbak ng padded sequence ang ipinakilala sa PyTorch. Halimbawa, mayroon tayong input na padded minibatch na ganito ang hitsura:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Dito, ang 0 ay kumakatawan sa mga padded na halaga, at ang aktwal na haba ng vector ng mga input sequence ay `[5,3,1]`.

Upang epektibong mag-train ng RNN gamit ang padded sequence, nais nating simulan ang training ng unang grupo ng RNN cells gamit ang malaking minibatch (`[1,6,9]`), ngunit pagkatapos ay tapusin ang pagproseso ng ikatlong sequence, at ipagpatuloy ang training gamit ang mas maiikling minibatch (`[2,7]`, `[3,8]`), at iba pa. Kaya, ang packed sequence ay kinakatawan bilang isang vector - sa ating halimbawa `[1,6,9,2,7,3,8,4,5]`, at isang length vector (`[5,3,1]`), mula rito madali nating maibabalik ang orihinal na padded minibatch.

Upang makagawa ng packed sequence, maaari nating gamitin ang function na `torch.nn.utils.rnn.pack_padded_sequence`. Lahat ng recurrent layers, kabilang ang RNN, LSTM, at GRU, ay sumusuporta sa packed sequences bilang input, at gumagawa ng packed output, na maaaring i-decode gamit ang `torch.nn.utils.rnn.pad_packed_sequence`.

Upang makagawa ng packed sequence, kailangan nating ipasa ang length vector sa network, kaya kailangan natin ng ibang function para maghanda ng mga minibatch:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Ang aktwal na network ay magiging halos katulad ng `LSTMClassifier` sa itaas, ngunit ang `forward` pass ay tatanggap ng parehong padded minibatch at ang vector ng haba ng mga sequence. Pagkatapos ng pag-compute ng embedding, iko-compute natin ang packed sequence, ipapasa ito sa LSTM layer, at pagkatapos ay i-unpack ang resulta pabalik.

> **Note**: Sa totoo lang, hindi natin ginagamit ang na-unpack na resulta na `x`, dahil ginagamit natin ang output mula sa mga hidden layer sa mga susunod na kalkulasyon. Kaya, maaari nating alisin ang pag-unpack nang tuluyan mula sa code na ito. Ang dahilan kung bakit inilagay natin ito dito ay upang madali mong mabago ang code na ito, sakaling kailanganin mong gamitin ang output ng network sa mga karagdagang kalkulasyon.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Tandaan:** Maaaring napansin mo ang parameter na `use_pack_sequence` na ipinapasa natin sa training function. Sa kasalukuyan, ang `pack_padded_sequence` function ay nangangailangan ng length sequence tensor na nasa CPU device, kaya't kailangang iwasan ng training function ang paglipat ng length sequence data sa GPU kapag nagte-training. Maaari mong tingnan ang implementasyon ng `train_emb` function sa file na [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## Bidirectional at Multilayer RNNs

Sa ating mga halimbawa, lahat ng recurrent networks ay gumagana sa isang direksyon lamang, mula simula ng isang sequence hanggang sa dulo. Mukhang natural ito, dahil kahalintulad ito ng paraan ng pagbabasa at pakikinig sa pagsasalita. Gayunpaman, dahil sa maraming praktikal na sitwasyon, may kakayahan tayong magkaroon ng random access sa input sequence, maaaring may kabuluhan na patakbuhin ang recurrent computation sa parehong direksyon. Ang ganitong mga network ay tinatawag na **bidirectional** RNNs, at maaari itong likhain sa pamamagitan ng pagpasa ng `bidirectional=True` na parameter sa RNN/LSTM/GRU constructor.

Kapag gumagamit ng bidirectional network, kakailanganin natin ng dalawang hidden state vectors, isa para sa bawat direksyon. Ang PyTorch ay nag-eencode ng mga vectors na ito bilang isang vector na may doble ang laki, na napaka-kombinyente, dahil karaniwan mong ipapasa ang resulting hidden state sa fully-connected linear layer, at kailangan mo lamang isaalang-alang ang pagtaas ng laki nito kapag gumagawa ng layer.

Ang recurrent network, one-directional man o bidirectional, ay nakakakuha ng ilang mga pattern sa loob ng isang sequence, at maaaring itago ang mga ito sa state vector o ipasa sa output. Tulad ng convolutional networks, maaari tayong magtayo ng isa pang recurrent layer sa ibabaw ng una upang makuha ang mas mataas na antas ng mga pattern, na binuo mula sa mga low-level pattern na nakuha ng unang layer. Ito ang nagdadala sa atin sa konsepto ng **multi-layer RNN**, na binubuo ng dalawa o higit pang recurrent networks, kung saan ang output ng nakaraang layer ay ipinapasa sa susunod na layer bilang input.

![Larawan na nagpapakita ng Multilayer long-short-term-memory- RNN](../../../../../translated_images/tl/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Larawan mula sa [napakagandang post na ito](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) ni Fernando López*

Ginagawang madali ng PyTorch ang pagbuo ng ganitong mga network, dahil kailangan mo lamang ipasa ang `num_layers` na parameter sa RNN/LSTM/GRU constructor upang awtomatikong makabuo ng ilang layers ng recurrence. Ito rin ay nangangahulugan na ang laki ng hidden/state vector ay tataas nang proporsyonal, at kailangan mo itong isaalang-alang kapag hinahandle ang output ng recurrent layers.


## RNNs para sa iba pang mga gawain

Sa yunit na ito, nakita natin na maaaring gamitin ang RNNs para sa sequence classification, ngunit sa katunayan, kaya rin nilang hawakan ang mas maraming gawain tulad ng pagbuo ng teksto, pagsasalin ng wika, at iba pa. Tatalakayin natin ang mga gawaing ito sa susunod na yunit.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
